In [ ]:
%matplotlib inline
import torch
import torchvision
from torch import nn
from torch.nn import functional as F
import sys
sys.path.append('..')
from tools import tool_pytorch_017 as ty

pretrained_net = torchvision.models.resnet18(pretrained=True)
list(pretrained_net.children())[-3:]

# 创建一个全卷积网络net。 它复制了ResNet-18中大部分的预训练层，除了最后的全局平均汇聚层和最接近输出的全连接层。
net = nn.Sequential(*list(pretrained_net.children())[:-2])

# 给定高度为320和宽度为480的输入，net的前向传播将输入的高和宽减小至原来的32分之1，即10和15
X = torch.rand(size=(1, 3, 320, 480))
net(X).shape

# 接下来使用1x1卷积层将输出通道数转换为Pascal VOC2012数据集的类数（21类）。 
# 最后需要将特征图的高度和宽度增加32倍，从而将其变回输入图像的高和宽。 
num_classes = 21
net.add_module('final_conv', nn.Conv2d(512, num_classes, kernel_size=1))
net.add_module('transpose_conv', nn.ConvTranspose2d(num_classes, num_classes,
                                    kernel_size=64, padding=16, stride=32))

def bilinear_kernel(in_channels, out_channels, kernel_size):
    '''双线性插值的上采样通过转置卷积层实现'''
    factor = (kernel_size + 1) // 2
    if kernel_size % 2 == 1:
        center = factor - 1
    else:
        center = factor - 0.5
    og = (torch.arange(kernel_size).reshape(-1, 1),
          torch.arange(kernel_size).reshape(1, -1))
    filt = (1 - torch.abs(og[0] - center) / factor) * \
           (1 - torch.abs(og[1] - center) / factor)
    weight = torch.zeros((in_channels, out_channels,
                          kernel_size, kernel_size))
    weight[range(in_channels), range(out_channels), :, :] = filt
    return weight

conv_trans = nn.ConvTranspose2d(3, 3, kernel_size=4, padding=1, stride=2,
                                bias=False)
conv_trans.weight.data.copy_(bilinear_kernel(3, 3, 4));

# 读取图像X，将上采样的结果记作Y。为了打印图像，我们需要调整通道维的位置。
# 转置卷积层将图像的高和宽分别放大了2倍。 
img = torchvision.transforms.ToTensor()(ty.Image.open('../data_sets/catdog.jpg'))
X = img.unsqueeze(0)
Y = conv_trans(X)
out_img = Y[0].permute(1, 2, 0).detach()

ty.set_figsize()
print('input image shape:', img.permute(1, 2, 0).shape)
ty.plt.imshow(img.permute(1, 2, 0));
print('output image shape:', out_img.shape)
ty.plt.imshow(out_img);

#  初始化参数
W = bilinear_kernel(num_classes, num_classes, 64)
net.transpose_conv.weight.data.copy_(W);

batch_size, crop_size = 32, (320, 480)
# 加载VOC数据集
train_iter, test_iter = ty.load_data_voc(batch_size, crop_size)

def loss(inputs, targets):
    '''损失函数和准确率计算与图像分类中的并没有本质上的不同'''
    return F.cross_entropy(inputs, targets, reduction='none').mean(1).mean(1)

def train_batch(net, X, y, loss, trainer, devices):
    """用多GPU进行小批量训练`"""
    if isinstance(X, list):
        # 微调BERT中所需（稍后讨论）
        X = [x.to(devices[0]) for x in X]
    else:
        X = X.to(devices[0])
    y = y.to(devices[0])
    net.train()
    trainer.zero_grad()
    pred = net(X)
    l = loss(pred, y)
    l.sum().backward()
    trainer.step()
    train_loss_sum = l.sum()
    train_acc_sum = tpy.accuracy(pred, y)
    return train_loss_sum, train_acc_sum

def train(net, train_iter, test_iter, loss, trainer, num_epochs,
               devices=tpy.try_all_gpus()):
    """用多GPU进行模型训练`"""
    timer, num_batches = tpy.Timer(), len(train_iter)
    animator = tpy.Animator(xlabel='epoch', xlim=[1, num_epochs], ylim=[0, 1],
                            legend=['train loss', 'train acc', 'test acc'])
    net = nn.DataParallel(net, device_ids=devices).to(devices[0])
    for epoch in range(num_epochs):
        # 4个维度：储存训练损失，训练准确度，实例数，特点数
        metric = tpy.Accumulator(4)
        for i, (features, labels) in enumerate(train_iter):
            timer.start()
            l, acc = train_batch(
                net, features, labels, loss, trainer, devices)
            metric.add(l, acc, labels.shape[0], labels.numel())
            timer.stop()
            if (i + 1) % (num_batches // 5) == 0 or i == num_batches - 1:
                animator.add(epoch + (i + 1) / num_batches,
                             (metric[0] / metric[2], metric[1] / metric[3],
                              None))
        test_acc = tpy.evaluate_accuracy_gpu(net, test_iter)
        animator.add(epoch + 1, (None, None, test_acc))
    print(f'loss {metric[0] / metric[2]:.3f}, train acc '
          f'{metric[1] / metric[3]:.3f}, test acc {test_acc:.3f}')
    print(f'{metric[2] * num_epochs / timer.sum():.1f} examples/sec on '
          f'{str(devices)}')
    
    
    
num_epochs, lr, wd, devices = 5, 0.001, 1e-3, ty.try_all_gpus()
trainer = torch.optim.SGD(net.parameters(), lr=lr, weight_decay=wd)
train(net, train_iter, test_iter, loss, trainer, num_epochs, devices)

def predict(img):
    '''在预测时，我们需要将输入图像在各个通道做标准化，并转成卷积神经网络所需要的四维输入格式。'''
    X = test_iter.dataset.normalize_image(img).unsqueeze(0)
    pred = net(X.to(devices[0])).argmax(dim=1)
    return pred.reshape(pred.shape[1], pred.shape[2])

def label2image(pred):
    '''为了可视化预测的类别给每个像素，我们将预测类别映射回它们在数据集中的标注颜色。'''
    colormap = torch.tensor(ty.VOC_COLORMAP, device=devices[0])
    X = pred.long()
    return colormap[X, :]

voc_dir = ty.download_extract('voc2012', 'VOCdevkit/VOC2012')
test_images, test_labels = ty.read_voc_images(voc_dir, False)
n, imgs = 4, []
for i in range(n):
    crop_rect = (0, 0, 320, 480)
    X = torchvision.transforms.functional.crop(test_images[i], *crop_rect)
    pred = label2image(predict(X))
    imgs += [X.permute(1,2,0), pred.cpu(),
             torchvision.transforms.functional.crop(
                 test_labels[i], *crop_rect).permute(1,2,0)]
ty.show_images(imgs[::3] + imgs[1::3] + imgs[2::3], 3, n, scale=2);

